In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from scipy.optimize import minimize, differential_evolution
from scipy.stats.qmc import Halton

import json
import os

In [2]:
potential = 'DW'
cutoff = 8
shots = 2048

In [3]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [4]:
min_eigenvalue

np.float64(0.8845804438664059)

In [5]:
cnot_pool = []
cz_pool = []

for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                cnot_pool.append(qml.CNOT(wires=[control, target]))
                cz_pool.append(qml.CZ(wires=[control, target]))

In [6]:
RY_pool = [qml.RY(0.0, wires=x) for x in range(num_qubits)]
RZ_pool = [qml.RZ(0.0, wires=x) for x in range(num_qubits)]
#RX_pool = [qml.RX(0.0, wires=x) for x in range(num_qubits)]
operator_pool = RZ_pool + cz_pool + RY_pool #+ RX_pool

In [7]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)

@qml.qnode(dev)
def circuit(params, trial_op, op_list):

    if len(op_list) > 0:
        for o, p, w, _ in op_list:
            if (o == qml.CNOT) | (o == qml.CZ):
                o(wires=w)
            else:
                o(p, wires=w)

    op = type(trial_op)

    if (type(trial_op) == qml.CNOT) | (type(trial_op) == qml.CZ):
        op(wires=trial_op.wires)
    else:
        op(params, wires=trial_op.wires)
  
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [8]:
def cost_function(params, trial_op, op_list):
    
    params = pnp.tensor(params, requires_grad=True)
    energy = circuit(params, trial_op, op_list)

    return energy

In [18]:
num_steps = 10

x0 = np.random.rand()

#variables
max_iter = 1
strategy = "randtobest1bin"
tol = 1e-2
abs_tol = 1e-2
popsize = 5

op_list = []

# Generate Halton sequence
num_dimensions = len(op_list) + 1
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

bounds = [(0, 2 * np.pi) for _ in range(1)]

for i in range(num_steps):

    print(f"step: {i}")

    energies = []
    e_params = []


    for trial_op in operator_pool:
        
        res = minimize(
                cost_function,
                x0,
                args=(trial_op, op_list),
                method= "COBYLA",
                options= {'maxiter':10000, 'tol': 1e-8}
            )
        '''
        res = differential_evolution(cost_function,
                                    bounds,
                                    args=(trial_op, op_list),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples
                                    )
        '''
        energies.append(res.fun)
        e_params.append(res.x)

    min_arg = np.argmin(energies)
    min_energy = energies[min_arg]
    print(f"Min energy: {min_energy}")

    min_op = type(operator_pool[min_arg])
    min_wires = operator_pool[min_arg].wires
    min_params = e_params[min_arg]

    if (i != 0):
        if np.abs(min_energy - op_list[-1][3]) < 1e-4:
            print("Converged")
            break

    if (i == 0):
        op_list.append((min_op, min_params, min_wires, min_energy))

    elif (i != 0) & (min_energy < op_list[-1][3]):
        op_list.append((min_op, min_params, min_wires, min_energy))
    elif (i != 0) & (min_energy >= op_list[-1][3]):
        print("Energy increased")
        break


    

step: 0


RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'

In [10]:
last_operator = {}
reduced_op_list = []
num_params = 0

for o, p, w, _ in op_list:

    if w[0] in last_operator.keys():
        if last_operator[w[0]] == o:
            print("Operator already added previously... skipping")
            continue

    if o == qml.CNOT:
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append(("CNOT", w.tolist()))

    elif o == qml.CZ:
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append(("CZ",w.tolist()))

    elif o == qml.RY:
        last_operator[w[0]] = o
        reduced_op_list.append(("RY",w.tolist()))
        num_params+=1

    elif o == qml.RZ:
        last_operator[w[0]] = o
        reduced_op_list.append(("RZ",w.tolist()))
        num_params+=1

    elif o == qml.RX:
        last_operator[w[0]] = o
        reduced_op_list.append(("RX",w.tolist()))
        num_params+=1

Operator already added previously... skipping


In [11]:
reduced_op_list

[('RY', [3]),
 ('CZ', [3, 0]),
 ('RY', [3]),
 ('RY', [2]),
 ('CZ', [2, 3]),
 ('RY', [1])]

In [12]:
@qml.qnode(dev)
def circuit(params):
    param_index=0
    for i in range(num_qubits-3, num_qubits):
        qml.RY(params[param_index], wires=i)
        param_index += 1

    # Apply entanglement
    for j in reversed(range(num_qubits-1, num_qubits)):
        qml.CZ(wires=[j - 1, j])

    # Apply RY rotations
    for k in range(num_qubits-2, num_qubits):
        qml.RY(params[param_index], wires=k)
        param_index += 1
    
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits))) 

In [13]:
params = np.random.random(size=5)
print(qml.draw(circuit)(params))

0: ────────────────────────┤ ╭<𝓗(M0)>
1: ──RY(0.15)──────────────┤ ├<𝓗(M0)>
2: ──RY(0.19)─╭●──RY(0.04)─┤ ├<𝓗(M0)>
3: ──RY(0.32)─╰Z──RY(0.84)─┤ ╰<𝓗(M0)>

M0 = 
[[ 2.375     +0.j  2.47487373+0.j  1.76776695+0.j  0.8660254 +0.j
   0.61237244+0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 2.47487373+0.j  5.875     +0.j  5.        +0.j  4.28660705+0.j
   1.73205081+0.j  1.36930639+0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 1.76776695+0.j  5.        +0.j 10.875     +0.j  7.96084166+0.j
   7.79422863+0.j  2.73861279+0.j  2.37170825+0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.8660254 +0.j  4.28660705+0.j  7.96084166+

In [14]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "COBYLA",
        "num steps": num_steps,
        "op_list": [str(x) for x in op_list],
        "reduced_op_list": reduced_op_list,
        "num_params": num_params}

In [15]:
data

{'potential': 'DW',
 'cutoff': 8,
 'optimizer': 'COBYLA',
 'num steps': 10,
 'op_list': ["(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, array([-0.78282526]), Wires([3]), np.float64(1.1406768574593928))",
  "(<class 'pennylane.ops.op_math.controlled_ops.CZ'>, array([0.74867031]), Wires([3, 0]), np.float64(1.096955447445147))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, array([-0.23588994]), Wires([3]), np.float64(1.0754864145959027))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, array([0.1545517]), Wires([2]), np.float64(1.0381623405134373))",
  "(<class 'pennylane.ops.op_math.controlled_ops.CZ'>, array([2.02967401]), Wires([2, 3]), np.float64(0.9755831493972225))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, array([-0.09727721]), Wires([1]), np.float64(0.9585713611832591))",
  "(<class 'pennylane.ops.op_math.controlled_ops.CZ'>, array([1.52967403]), Wires([2, 0]), np.float64(0.936347013731049))"],
 'red

In [16]:
params = np.random.uniform(0, 2 * np.pi, size=num_params)

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def construct_ansatz(reduced_op_list, params):

    params_index = 0

    for o, w in reduced_op_list:
        if o == "CNOT":
            qml.CNOT(wires=w)
        elif o == "CZ":
            qml.CZ(wires=w)
        elif o == "RY":
            qml.RY(params[params_index], wires=w)
            params_index +=1

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [17]:
print(qml.draw(construct_ansatz, show_matrices=False)(reduced_op_list, params))

0: ───────────╭Z──────────────┤ ╭<𝓗(M0)>
1: ───────────│───RY(4.36)────┤ ├<𝓗(M0)>
2: ───────────│───RY(5.17)─╭●─┤ ├<𝓗(M0)>
3: ──RY(3.82)─╰●──RY(4.58)─╰Z─┤ ╰<𝓗(M0)>


In [ ]:
# Save the variable to a JSON file
base_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\NoRotFiles\\"
os.makedirs(base_path, exist_ok=True)

cd = qml.draw(construct_ansatz, show_matrices=False)(reduced_op_list, params)

path = base_path + "cd_{}_{}.txt".format(potential, cutoff)
with open(path, 'w', encoding='utf-8') as f:
    f.write(cd)

In [ ]:
# Save the variable to a JSON file
base_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\NoRotFiles\\"
#base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [ ]:
reduced_op_list

In [ ]:
qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=num_qubits)

In [ ]:
last_operator = {}
reduced_op_list = []
num_params = 0

for o, p, w, _ in op_list:

    if (o == qml.CNOT) | (o == qml.CZ):
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append((o,w))

    elif w[0] in last_operator.keys():
        if last_operator[w[0]] == o:
            print("Operator already added previously... skipping")
            continue
        else:
            last_operator[w[0]] = o
            reduced_op_list.append((o,w))
            num_params = num_params + 3
    else:
        last_operator[w[0]] = o
        reduced_op_list.append((o,w))
        num_params = num_params + 3

In [ ]:
params = np.random.uniform(0, 2 * np.pi, size=num_params)

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def construct_ansatz(reduced_op_list, params):

    params_index = 0

    for i, (o, w) in enumerate(reduced_op_list):
        if (o == qml.CNOT) | (o == qml.CZ):
            o(wires=w)
        else:
            num_gate_params = o.num_params
            o(*params[params_index:(params_index + num_gate_params)], wires=w)
            params_index = params_index + num_gate_params

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))